In [1]:
from math import cos, sin, sqrt
import os

import numpy as np

# POLIASTRO_TARGET can be set to `cpu`, `parallel` or `cuda`. Default: `cpu`
os.environ['POLIASTRO_TARGET'] = 'cpu'  

# POLIASTRO_INLINE can be set to `always` or `never`. Default: `never`
os.environ['POLIASTRO_INLINE'] = 'always'

from poliastro.core.jit import gjit, hjit, vjit, TARGET, INLINE

print(TARGET, INLINE)  # verfication

cpu always


In [2]:
@hjit('M(M)')
def transpose_M_(a):
    return (
        (a[0][0], a[1][0], a[2][0]),
        (a[0][1], a[1][1], a[2][1]),
        (a[0][2], a[1][2], a[2][2]),
    )

@hjit('M(M,M)')
def matmul_MM_(a, b):
    return (
        (
            a[0][0] * b[0][0] + a[0][1] * b[1][0] + a[0][2] * b[2][0],
            a[0][0] * b[0][1] + a[0][1] * b[1][1] + a[0][2] * b[2][1],
            a[0][0] * b[0][2] + a[0][1] * b[1][2] + a[0][2] * b[2][2],
        ),
        (
            a[1][0] * b[0][0] + a[1][1] * b[1][0] + a[1][2] * b[2][0],
            a[1][0] * b[0][1] + a[1][1] * b[1][1] + a[1][2] * b[2][1],
            a[1][0] * b[0][2] + a[1][1] * b[1][2] + a[1][2] * b[2][2],
        ),
        (
            a[2][0] * b[0][0] + a[2][1] * b[1][0] + a[2][2] * b[2][0],
            a[2][0] * b[0][1] + a[2][1] * b[1][1] + a[2][2] * b[2][1],
            a[2][0] * b[0][2] + a[2][1] * b[1][2] + a[2][2] * b[2][2],
        ),
    )

@hjit('V(V,M)')
def matmul_VM_(a, b):
    return (
        a[0] * b[0][0] + a[1] * b[1][0] + a[2] * b[2][0],
        a[0] * b[0][1] + a[1] * b[1][1] + a[2] * b[2][1],
        a[0] * b[0][2] + a[1] * b[1][2] + a[2] * b[2][2],
    )

@hjit('M(f,u1)')
def rotation_matrix_(angle, axis):
    c = cos(angle)
    s = sin(angle)
    if axis == 0:
        return (
            (1.0, 0.0, 0.0),
            (0.0,   c,  -s),
            (0.0,   s,   c),
        )
    if axis == 1:
        return (
            (  c, 0.0,   s),
            (0.0, 1.0, 0.0),
            (  s, 0.0,   c),
        )
    if axis == 2:
        return (
            (  c,  -s, 0.0),
            (  s,   c, 0.0),
            (0.0, 0.0, 1.0),
        )
    raise ValueError("Invalid axis: must be one of 0, 1 or 2")

@hjit('Tuple([V,V])(f,f,f,f)')
def rv_pqw_(k, p, ecc, nu):
    sinnu = sin(nu)
    cosnu = cos(nu)
    a = p / (1 + ecc * cosnu)
    b = sqrt(k / p)
    return (
        (cosnu * a, sinnu * a, 0),
        (-sinnu * b, (ecc + cosnu) * b, 0),
    )

@hjit('M(f,f,f)')
def coe_rotation_matrix_(inc, raan, argp):
    r = rotation_matrix_(raan, 2)
    r = matmul_MM_(r, rotation_matrix_(inc, 0))
    return matmul_MM_(r, rotation_matrix_(argp, 2))

@hjit('Tuple([V,V])(f,f,f,f,f,f,f)')
def coe2rv_(k, p, ecc, inc, raan, argp, nu):
    "Converts from classical orbital to state vectors"
    r, v = rv_pqw_(k, p, ecc, nu)
    rm = transpose_M_(coe_rotation_matrix_(inc, raan, argp))
    return matmul_VM_(r, rm), matmul_VM_(v, rm)